## コーパスのダウンロード

[JParaCrawl](https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/)のサイトからコーパスデータをダウンロードします。

In [2]:
!mkdir data

In [ ]:
%cd data

In [ ]:
!wget https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/3.0/bitext/en-ja.tar.gz

## コーパスの前処理

OpenNMT-pyで扱いやすいようにデータを前処理します。
まずは、データを展開します。

In [ ]:
!tar xvf en-ja.tar.gz

In [ ]:
%cd en-ja/

データの中身を確認してみましょう。タブ区切りで5つのフィールドがあり、4、5番目にそれぞれ英文、和文がありました。

文数は25,740,835でした。

In [12]:
!head en-ja.bicleaner05.txt

0001vip.cocolog-nifty.com	0001vip.cocolog-nifty.com	0.535	And everyone will not care that it is not you.	鼻・口のところはあらかじめ少し切っておくといいですね。
0001vip.cocolog-nifty.com	0001vip.cocolog-nifty.com	0.557	And everyone will not care that it is not you.	アドレス置いとくので、消されないうちにメールくれたら嬉しいです。
000-lhr.web.wox.cc	000-lhr.web.wox.cc	0.743	Sponsored link This advertisement is displayed when there is no update for a certain period of time.	スポンサードリンク この広告は一定期間更新がない場合に表示されます。
000-lhr.web.wox.cc	000-lhr.web.wox.cc	0.750	Also, it will always be hidden when becoming a premium user .	また、 プレミアムユーザー になると常に非表示になります。
000-lhr.web.wox.cc	000-lhr.web.wox.cc	0.751	It will return to non-display when content update is done.	コンテンツの更新が行われると非表示に戻ります。
000-lhr.web.wox.cc	kapuri21.web.wox.cc	0.743	Sponsored link This advertisement is displayed when there is no update for a certain period of time.	スポンサードリンク この広告は一定期間更新がない場合に表示されます。
000-lhr.web.wox.cc	kapuri21.web.wox.cc	0.750	Also, it will always be hidden when becoming a premium user 

In [22]:
!wc -l en-ja.bicleaner05.txt

25740835 en-ja.bicleaner05.txt


英文と和文だけを取り出します。

In [16]:
!cat en-ja.bicleaner05.txt | cut -f4 > en.txt
!cat en-ja.bicleaner05.txt | cut -f5 > ja.txt

## 文をトークン化

学習用に文章をトークン化します。

トークン化には[MeCab](https://taku910.github.io/mecab/)等が用いられることも多いのですが、ここでは、[SentencePiece](https://github.com/google/sentencepiece)を使ってトークン化してみます。

まず、Pythonのパッケージをインストールします。

In [ ]:
!pip install sentencepiece

全部の文章から単語分割モデルを学習しようとすると、筆者のPCのメモリ(8GB)では足りなかったので、一部の文(100,000文)から学習しました。

PCの環境によっては文数を増やしてもよいかもしれません。

まず最初に、英文、和文の中身の一部をシャッフルして取り出します。

In [2]:
!shuf -n 100000 en.txt -o vocab_train.en

In [10]:
!shuf -n 100000 ja.txt -o vocab_train.ja

取り出した文で学習します。

character_coverageは、英語の場合は公式サイトで例示されている値、日本語の場合は一般的によいと言われている値です。

In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train(
   '--input=vocab_train.en --model_prefix=sentencepiece_en --vocab_size=32000 --character_coverage=0.98'
)

In [ ]:
spm.SentencePieceTrainer.Train(
   '--input=vocab_train.ja --model_prefix=sentencepiece_ja --vocab_size=32000 --character_coverage=0.9995'
)

学習したモデルを使って、試しにトークン化してみます。

英語の場合は空白で区切っただけのように見えますが、日本語の場合は文法通りではなく独自に学習しているようです。

In [12]:
sp_en = spm.SentencePieceProcessor("sentencepiece_en.model")

In [13]:
print(sp_en.encode("It will return to non-display when content update is done.", out_type=str))

['▁It', '▁will', '▁return', '▁to', '▁non', '-', 'display', '▁when', '▁content', '▁update', '▁is', '▁done', '.']


In [14]:
sp_ja = spm.SentencePieceProcessor("sentencepiece_ja.model")

In [ ]:
print(sp_ja.encode("スポンサードリンク この広告は一定期間更新がない場合に表示されます。", out_type=str))

['▁', 'スポンサー', 'ド', 'リンク', '▁この', '広告', 'は', '一定', '期間', '更新', 'がない場合', 'に表示されます', '。']


In [19]:
print(' '.join(sp_ja.encode("スポンサードリンク この広告は一定期間更新がない場合に表示されます。", out_type=str)))

▁ スポンサー ド リンク ▁この 広告 は 一定 期間 更新 がない場合 に表示されます 。


コーパスのデータとトークン化したファイルを作成します。

このファイルを使って、翻訳モデルを作成します。

In [25]:
with open("./en.txt") as in_f:
    with open("en_tokenized.txt", mode='w') as out_f:
        for line in in_f:
            out_f.write(' '.join(sp_en.encode(line, out_type=str)) + "\n")

In [26]:
with open("./ja.txt") as in_f:
    with open("ja_tokenized.txt", mode='w') as out_f:
        for line in in_f:
            out_f.write(' '.join(sp_ja.encode(line, out_type=str)) + "\n")

トークン化したら、そのファイルの一部(25730000文)を訓練用、5000文を検証用、5000文をテスト用に分けます。

コーパスファイルの頭から何万文と言うように分けると、取得元のドメインが偏るのでシャッフルしてデータを分けるようにします。

まず最初にコーパスの文数までの数値がランダムにならんだファイルを作成します。

In [19]:
!seq `wc -l en-ja.bicleaner05.txt | cut -f1 -d' '` | shuf -o line_nums.txt -

In [ ]:
訓練用、検証用、テスト用の文に使うデータの行番号が入ったファイルを作成します。

In [49]:
!head --lines=25730000 line_nums.txt | sort --numeric-sort > train_line_nums.txt
!head --lines=25735000 line_nums.txt | tail --lines=5000 | sort --numeric-sort > val_line_nums.txt
!head --lines=25740000 line_nums.txt | tail --lines=5000 | sort --numeric-sort > test_line_nums.txt

文のファイル名と、行番号のファイル名から、行番号の文を取り出す関数を定義して、訓練用、検証用、テスト用のファイルを作成します。

In [37]:
# input_file: 取り出し元ファイル
# num_fileの行番号の行だけ取り出します。
# output_file: 取り出した結果ファイル
def extract_lines(input_file, num_file, output_file):
    text_line_num = 1
    with open(num_file) as line_f:
        line_num = int(line_f.readline())
        with open(input_file) as in_f:
            with open(output_file, mode='w') as out_f:
                for line in in_f:
                    if text_line_num == line_num:
                        out_f.write(line)
                        line_num = line_f.readline()
                        if line_num == '':
                            break
                        else:
                            line_num = int(line_num)
                    text_line_num += 1

In [50]:
extract_lines("en_tokenized.txt", "train_line_nums.txt", "train.en")
extract_lines("ja_tokenized.txt", "train_line_nums.txt", "train.ja")
extract_lines("en_tokenized.txt", "val_line_nums.txt", "val.en")
extract_lines("ja_tokenized.txt", "val_line_nums.txt", "val.ja")
extract_lines("en_tokenized.txt", "test_line_nums.txt", "test.en")
extract_lines("ja_tokenized.txt", "test_line_nums.txt", "test.ja")

試しに日本語の訓練データを見てみましょう。

In [51]:
!head train.ja

▁ 鼻 ・ 口 の ところ は あらかじめ 少し 切 って お く と いい ですね 。
▁ アドレス 置 い と く ので 、 消 されない うち に メール く れた ら 嬉しい です 。
▁ スポンサー ド リンク ▁この 広告 は 一定 期間 更新 がない場合 に表示されます 。
▁また 、 ▁ プレミアム ユーザー ▁ になると 常に 非表示 になります 。
▁ コンテンツ の 更新 が 行われる と 非表示 に戻ります 。
▁ スポンサー ド リンク ▁この 広告 は 一定 期間 更新 がない場合 に表示されます 。
▁また 、 ▁ プレミアム ユーザー ▁ になると 常に 非表示 になります 。
▁ コンテンツ の 更新 が 行われる と 非表示 に戻ります 。
▁You tu be を中心に ミニ マ リスト と言って いる 方 の 動画 を たくさん み ました が 、 納 得 いく ものの も 多く 、 不 況 と 少 子 化 を 呼ばれ る ” 今 の ” 日本 には 合 っている 考え方 なのか な 、 と 感じ ています 。
▁f f mp e g ▁- i ▁ sa mp le . mp 4 ▁- str ic t ▁ -2 ▁ v ide o . web m ▁ まとめ ▁Web 上で 動画 を 設置 するとき は 、 Y out ub e に アップ して 埋め 込む 方法 ばかり 使 っていました が 、 これ で 複数の 動画 形式 を作る ことができる ので 、 自分の サーバ に 設定 することも 可能 になります ね 。


In [ ]:
%cd ../..

## OpenNMT-pyで学習

データの準備ができたので、OpenNMT-pyで学習させます。

まずはOpenNMT-pyをインストールします。

In [ ]:
!pip install OpenNMT-py

以下のようなYamlファイルを作成します。

ボキャブラリー(語彙)ファイルを作成します。

In [ ]:
!onmt_build_vocab -config src/train_en_ja.yml -n_sample 10000

以下のようなシェルスクリプトを作成します。

以下のようなDockerファイルを作成します。

以下のようなスクリプトを使ってDockerイメージを作成し、Amazon Elastic Container Registry(ECR)にイメージをpushします。

In [ ]:
!./build_and_push.sh jparacrawl-train

前に作成したIAMロールを取得します。

In [1]:
import boto3

role_name = "SageMaker-local"

iam = boto3.client("iam")
role = iam.get_role(RoleName=role_name)["Role"]["Arn"]

SageMakerのセッションを開始します。

In [3]:
import sagemaker as sage

sess = sage.Session()

In [10]:
prefix = 'jparacrawl/training'
WORK_DIRECTORY = 'data_train'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)
print("data uploaded.")

data uploaded.


In [15]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/jparacrawl-train:latest'.format(account, region)

estimator = sage.estimator.Estimator(image,
                       role, 1, 'ml.g4dn.12xlarge',
                       output_path="s3://{}/jparacrawl/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

print("estimation starting.")
estimator.fit({"training": data_location})

estimation starting.
2022-09-27 12:26:48 Starting - Starting the training job...
2022-09-27 12:27:11 Starting - Preparing the instances for trainingProfilerReport-1664281607: InProgress
......
2022-09-27 12:28:21 Downloading - Downloading input data.................Tue Sep 27 12:31:18 UTC 2022
/opt/ml/input/:
total 8
drw-r--r-- 2 root root 4096 Sep 27 12:28 config
drwxr-xr-x 3 root root 4096 Sep 27 12:28 data
/opt/ml/input/config:
total 28
-rw-r--r-- 1 root root 217 Sep 27 12:28 debughookconfig.json
-rw-r--r-- 1 root root   2 Sep 27 12:28 hyperparameters.json
-rw-r--r-- 1 root root 502 Sep 27 12:28 init-config.json
-rw-r--r-- 1 root root 107 Sep 27 12:28 inputdataconfig.json
-rw-r--r-- 1 root root   2 Sep 27 12:28 metric-definition-regex.json
-rw-r--r-- 1 root root 220 Sep 27 12:28 profilerconfig.json
-rw-r--r-- 1 root root 280 Sep 27 12:28 resourceconfig.json
/opt/ml/input/data:
total 8
drwxr-xr-x 3 root root 4096 Sep 27 12:28 training
-rw-r--r-- 1 root root  478 Sep 27 12:28 training